# Strategy Rules

- Determine Upward Trend: Identify an upward trend by checking if the closing price is above a moving average (e.g., EMA).
- Check for Bearish Volatility: Ensure there is no bearish volatility by analyzing the ATR indicators.
- Entry Signal: Generate an entry signal if there is an upward trend and no bearish volatility. Exit Signal: Implement a trailing stop-loss to generate an exit signal and protect profits.
- Exit Signal: Implement a trailing stop-loss to generate an exit signal and protect profits.
- tight trailing stop when if intrade and bearish volatility.


In [296]:
import pandas as pd
import numpy as np
import vectorbt as vbt
from datetime import datetime
import os 
from numba import njit
from vectorbt.portfolio.enums import Direction, SizeType
import talib
import requests
import time


symbol = "BTC-USDT"
timeframe = "1day"
market_type = "spot"
start_time = "2021-01-01 00:00:00"
end_time = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")


skip_fetch = False


In [297]:

if not skip_fetch:
    # print("Fetching data...")
    def fetch_kucoin_candles_chunk(symbol, market_type="spot", timeframe="1min", start_time=None, end_time=None):
        time.sleep(0.2)  # Rate limiting
        base_url = "https://api.kucoin.com" if market_type.lower() == "spot" else "https://api-futures.kucoin.com"
        url = base_url + "/api/v1/market/candles"
        params = {"type": timeframe, "symbol": symbol.upper()}
        if start_time:
            params["startAt"] = int(time.mktime(time.strptime(start_time, "%Y-%m-%d %H:%M:%S")))
        if end_time:
            params["endAt"] = int(time.mktime(time.strptime(end_time, "%Y-%m-%d %H:%M:%S")))
        try:
            resp = requests.get(url, params=params, timeout=10)
            resp.raise_for_status()
            data = resp.json()
            if data.get("code") == "200000":
                return data["data"]
            else:
                raise Exception(f"KuCoin API error: {data}")
        except requests.exceptions.RequestException as e:
            raise Exception(f"Request failed: {str(e)}")

    def fetch_all_kucoin_candles(symbol, market_type="spot", timeframe="1min", start_time=None, end_time=None):
        chunks = []
        current_end = end_time
        start_timestamp = int(time.mktime(time.strptime(start_time, "%Y-%m-%d %H:%M:%S")))
        
        while True:
            try:
                chunk = fetch_kucoin_candles_chunk(symbol, market_type, timeframe, start_time, current_end)
                if not chunk:
                    print("No more data available")
                    break
                earliest_ts = int(chunk[-1][0])
                print(f"Fetched {len(chunk)} candles from {datetime.fromtimestamp(earliest_ts)}")
                chunks.extend(chunk)
                if earliest_ts <= start_timestamp:
                    print("Reached start time")
                    break
                current_end = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(earliest_ts - 60))
            except Exception as e:
                print(f"Error occurred: {str(e)}")
                break
        
        if not chunks:
            return []
        
        chunks.sort(key=lambda x: x[0])  # Sort by timestamp
        result = [candle for candle in chunks if start_timestamp <= int(candle[0]) <= int(time.mktime(time.strptime(end_time, "%Y-%m-%d %H:%M:%S")))]
        print(f"Total candles fetched: {len(result)}")
        return result


    data = fetch_all_kucoin_candles(symbol, market_type, timeframe, start_time, end_time)
    data
else:
    print("Skipping fetch...")

Fetched 1466 candles from 2021-01-01 00:00:00
Reached start time
Total candles fetched: 1466


In [298]:

# Assuming 'data' is the list of candlestick data
df = pd.DataFrame(data)
df[['timestamp', 'open', 'close', 'high', 'low']] = df[[0, 1, 2, 3, 4]]
df.drop([0, 1, 2, 3, 4, 5, 6], axis=1, inplace=True)

# Explicitly cast 'timestamp' to numeric type
df['timestamp'] = pd.to_numeric(df['timestamp'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', utc=True)

df.set_index('timestamp', inplace=True)
df.sort_index(inplace=True)
df[['open', 'close', 'high', 'low']] = df[['open', 'close', 'high', 'low']].astype(float)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1466 entries, 2021-01-01 00:00:00+00:00 to 2025-01-05 00:00:00+00:00
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    1466 non-null   float64
 1   close   1466 non-null   float64
 2   high    1466 non-null   float64
 3   low     1466 non-null   float64
dtypes: float64(4)
memory usage: 57.3 KB


,open,close,high,low
timestamp,,,,
2021-01-01 00:00:00+00:00,28924.9,29333.2,29607.1,28633.0
2021-01-02 00:00:00+00:00,29333.2,32185.4,33286.8,28949.2
2021-01-03 00:00:00+00:00,32181.3,33000.4,34766.0,31500.0
2021-01-04 00:00:00+00:00,33005.9,31993.0,33600.2,27500.0
2021-01-05 00:00:00+00:00,31993.1,33938.6,34371.0,29806.1
...,...,...,...,...
2025-01-01 00:00:00+00:00,93574.4,94584.1,95150.0,92900.8
2025-01-02 00:00:00+00:00,94584.2,96983.8,97839.1,94380.5
2025-01-03 00:00:00+00:00,96983.9,98173.9,98976.9,96105.4


## creating indicator to set signals long ONLY for simplicity

In [299]:
import pandas as pd
import numpy as np
import vectorbt as vbt


skip_cell = False

if not skip_cell:
    print("This cell is executed.")


    def momentum_strategy(close, high, low, atr_length, ema_length,vola_multiplier, vol_window):
        # Calculate EMA
        EMA = vbt.IndicatorFactory.from_talib('EMA')
        ema = EMA.run(close, timeperiod=ema_length).real.to_numpy()

        # Calculate ATR
        ATR = vbt.IndicatorFactory.from_talib('ATR')
        atr = ATR.run(high, low, close, timeperiod=atr_length).real.to_numpy()

        def rolling_max(arr, window=vol_window):
            result_rolling_max = np.full_like(arr, np.nan)
            for i in range(window-1, len(arr)):
                result_rolling_max[i] = np.max(arr[i-window+1:i+1])
            return result_rolling_max

        def compute_is_bearish_vol(high, low, atr, window=vol_window):
            rm = rolling_max(high, window=window)
            return (rm - low) > (atr * vola_multiplier)
        
        rm = rolling_max(high, window=vol_window)

        # Calculate conditions
        is_bullish = close > ema
        is_bearish_vol = compute_is_bearish_vol(high, low, atr)
        
        is_caution = is_bullish & (is_bearish_vol | (close < ema))

        signal_buy = is_bullish & ~is_caution

        conditions = [
            signal_buy,
            is_caution
        ]

        values = [10, 5]
        
        # Use np.select to create final signal
        signal = np.select(conditions, values, default=0)
        
        # Ensure signal has the same shape as input
        return signal,rm,is_bearish_vol  #.reshape(close.shape)

#     # Create the indicator factory
#     momentum_indicator = vbt.IndicatorFactory(
#         class_name='MomentumStrategy',
#         short_name='momentum',
#         input_names=['Close', 'High', 'Low'],
#         param_names=['atr_length', 'ema_length', 'vola_multiplier', 'vol_window'],
#         output_names=['signal']
#     ).from_apply_func(momentum_strategy)

#     # Define parameter ranges
#     atr_length = np.arange(4, 10, 1)  # Range from 5 to 10 with step 1
#     ema_length = np.arange(100, 241, 20)  # Range from 10 to 30 with step 5
#     vola_multiplier = [1.3,1.4,1.5,1.6,1.7,1.8]#np.arange(1, 2, 0.1)  # Range from 1 to 3 with step 0.1
#     vol_window = np.arange(4, 8, 1)  # Range from 5 to 15 with step 1

#     atr_length = 8
#     ema_length = 160
#     vola_multiplier = 1.8
#     vol_window = [7] # Range from 5 to 15 with step 1

#     # Run the indicator
#     signal = momentum_indicator.run(
#         df['close'],df['high'],df['low'],
#         atr_length=atr_length,
#         ema_length=ema_length,
#         vola_multiplier=vola_multiplier,
#         vol_window=vol_window,
#         param_product=True  
#     )
#     print(signal.signal.shape)
#     df_indicator=signal.signal


# # momentum_strategy(df['close'],df['high'],df['low'],8,160,1.8,7)

# # else:
# #     print("This cell is skipped.")

This cell is executed.


In [300]:
x = momentum_strategy(df['close'],df['high'],df['low'],8,160,1.8,7)
x[0]
x[1]
x[2]

timestamp
2021-01-01 00:00:00+00:00    False
2021-01-02 00:00:00+00:00    False
2021-01-03 00:00:00+00:00    False
2021-01-04 00:00:00+00:00    False
2021-01-05 00:00:00+00:00    False
                             ...  
2025-01-01 00:00:00+00:00     True
2025-01-02 00:00:00+00:00    False
2025-01-03 00:00:00+00:00    False
2025-01-04 00:00:00+00:00    False
2025-01-05 00:00:00+00:00    False
Name: low, Length: 1466, dtype: bool

In [301]:
df = df.copy()
df['signal'] = x[0]
df['rolling_max_np'] = x[1]
df['rolling_max_pd'] = df['high'].rolling(window=7).max()
df['is_bearish_vol'] = x[2]
ema_slow = talib.EMA(df['close'], timeperiod=200)
ema_fast = talib.EMA(df['close'], timeperiod=20)
df['ema_slow'] = ema_slow
df['ema_fast'] = ema_fast
df[['rolling_max_np','rolling_max_pd']].tail(40)

df[df['rolling_max_np']!=df['rolling_max_pd']]
df['atr'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=8)

df[(df.index >= '2023-01-15') & (df.index <= '2023-01-25')]['atr']




timestamp
2023-01-15 00:00:00+00:00    623.569204
2023-01-16 00:00:00+00:00    648.848054
2023-01-17 00:00:00+00:00    664.829547
2023-01-18 00:00:00+00:00    736.125854
2023-01-19 00:00:00+00:00    711.560122
2023-01-20 00:00:00+00:00    857.890107
2023-01-21 00:00:00+00:00    869.766343
2023-01-22 00:00:00+00:00    858.145550
2023-01-23 00:00:00+00:00    834.739857
2023-01-24 00:00:00+00:00    820.234875
2023-01-25 00:00:00+00:00    904.555515
Name: atr, dtype: float64

In [302]:
import plotly.graph_objects as go

buy = df_indicator[df_indicator == 10]
caution = df_indicator[df_indicator == 5]
none = df_indicator[df_indicator == 0]

fig = go.Figure()
fig.add_trace(go.Candlestick(x=df.index,open=df['open'],
                             close=df['close'],
                             high=df['high'],
                             low=df['low'],
                             name='candlestick'))

fig.update_layout(title='Chart',
                  xaxis_title = 'date',
                  yaxis_title='price',
                  xaxis_rangeslider_visible=False,
                  hovermode='x unified')
# fig.add_trace(go.Scatter(x=buy.index, y=buy, mode='markers', name='buy', marker=dict(color='green')))
fig.add_trace(go.Scatter(x=buy.index, y=buy, mode='markers',name='buy', marker=dict(color='green')))
fig.add_trace(go.Scatter(x=caution.index, y=caution, mode='markers',name='caution', marker=dict(color='yellow')))
fig.add_trace(go.Scatter(x=none.index, y=none, mode='markers',name='none', marker=dict(color='red')))

fig.add_trace(go.Scatter(x=df.index,y=ema_slow,mode='lines',name='ema_slow',line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df.index,y=ema_fast,mode='lines',name='ema_fast',line=dict(color='orange')))



fig.show()

In [303]:
def repeat_series_horizontally(series, target_shape):
    data = series.values.reshape(-1, 1)  # Convert to column vector
    return np.repeat(data, target_shape[1], axis=1)

nedded_copies = len(df_indicator.columns)
print('needed columns: ',nedded_copies)
print('needed rows: ',len(df_indicator))
print('shape:', df_indicator.shape)
print()

# copying the close price to the shape of the indicator
print('original shape:\n\n', df['close'])
dublicate_close = repeat_series_horizontally(df['close'], df_indicator.shape)
df_dublicate_close  = pd.DataFrame(dublicate_close,df.index)
df_dublicate_close 



AttributeError: 'Series' object has no attribute 'columns'

## compare the shape of the copied closing prices 

In [173]:
df_custom_indiator = df_indicator

if df_custom_indiator.shape == df_dublicate_close.shape:
    df_renamed_close = df_dublicate_close
    df_renamed_close.columns = df_custom_indiator.columns
    print("Columns have been renamed ")
else:
    print("The DataFrames do not have the same shape.")

# Display the resulting DataFrame
df_renamed_close.head()
df_renamed_close

Columns have been renamed 


momentum_atr_length              4                                      \
momentum_ema_length            100                                       
momentum_vola_multiplier       1.3                                 1.4   
momentum_vol_window              4        5        6        7        4   
timestamp                                                                
2021-01-01 00:00:00+00:00  29333.2  29333.2  29333.2  29333.2  29333.2   
2021-01-02 00:00:00+00:00  32185.4  32185.4  32185.4  32185.4  32185.4   
2021-01-03 00:00:00+00:00  33000.4  33000.4  33000.4  33000.4  33000.4   
2021-01-04 00:00:00+00:00  31993.0  31993.0  31993.0  31993.0  31993.0   
2021-01-05 00:00:00+00:00  33938.6  33938.6  33938.6  33938.6  33938.6   
...                            ...      ...      ...      ...      ...   
2025-01-01 00:00:00+00:00  94584.1  94584.1  94584.1  94584.1  94584.1   
2025-01-02 00:00:00+00:00  96983.8  96983.8  96983.8  96983.8  96983.8   
2025-01-03 00:00:00+00:00  98173.9  98173.9  98173.9  98173.9  98173.9   
2025-01-04 00:00:00+00:00  98215.4  98215.4  98215.4  98215.4  98215.4   
2025-01-05 00:00:00+00:00  97667.7  97667.7  97667.7  97667.7  97667.7   

momentum_atr_length                                                     ...  \
momentum_ema_length                                                     ...   
momentum_vola_multiplier                                  1.5           ...   
momentum_vol_window              5        6        7        4        5  ...   
timestamp                                                               ...   
2021-01-01 00:00:00+00:00  29333.2  29333.2  29333.2  29333.2  29333.2  ...   
2021-01-02 00:00:00+00:00  32185.4  32185.4  32185.4  32185.4  32185.4  ...   
2021-01-03 00:00:00+00:00  33000.4  33000.4  33000.4  33000.4  33000.4  ...   
2021-01-04 00:00:00+00:00  31993.0  31993.0  31993.0  31993.0  31993.0  ...   
2021-01-05 00:00:00+00:00  33938.6  33938.6  33938.6  33938.6  33938.6  ...   
...                            ...      ...      ...      ...      ...  ...   
2025-01-01 00:00:00+00:00  94584.1  94584.1  94584.1  94584.1  94584.1  ...   
2025-01-02 00:00:00+00:00  96983.8  96983.8  96983.8  96983.8  96983.8  ...   
2025-01-03 00:00:00+00:00  98173.9  98173.9  98173.9  98173.9  98173.9  ...   
2025-01-04 00:00:00+00:00  98215.4  98215.4  98215.4  98215.4  98215.4  ...   
2025-01-05 00:00:00+00:00  97667.7  97667.7  97667.7  97667.7  97667.7  ...   

momentum_atr_length              9                                      \
momentum_ema_length            240                                       
momentum_vola_multiplier       1.6               1.7                     
momentum_vol_window              6        7        4        5        6   
timestamp                                                                
2021-01-01 00:00:00+00:00  29333.2  29333.2  29333.2  29333.2  29333.2   
2021-01-02 00:00:00+00:00  32185.4  32185.4  32185.4  32185.4  32185.4   
2021-01-03 00:00:00+00:00  33000.4  33000.4  33000.4  33000.4  33000.4   
2021-01-04 00:00:00+00:00  31993.0  31993.0  31993.0  31993.0  31993.0   
2021-01-05 00:00:00+00:00  33938.6  33938.6  33938.6  33938.6  33938.6   
...                            ...      ...      ...      ...      ...   
2025-01-01 00:00:00+00:00  94584.1  94584.1  94584.1  94584.1  94584.1   
2025-01-02 00:00:00+00:00  96983.8  96983.8  96983.8  96983.8  96983.8   
2025-01-03 00:00:00+00:00  98173.9  98173.9  98173.9  98173.9  98173.9   
2025-01-04 00:00:00+00:00  98215.4  98215.4  98215.4  98215.4  98215.4   
2025-01-05 00:00:00+00:00  97667.7  97667.7  97667.7  97667.7  97667.7   

momentum_atr_length                                                     
momentum_ema_length                                                     
momentum_vola_multiplier                1.8                             
momentum_vol_window              7        4        5        6        7  
timestamp                                                               


In [174]:
atr_values = talib.ATR(df['high'], df['low'], df['close'], timeperiod=7).to_numpy().flatten()
print(type(atr_values))
print(atr_values.shape)

<class 'numpy.ndarray'>
(1466,)


In [175]:
# Numba-compiled order function
@njit
def order_func_nb(c, high, low, open_, entries, sl_prices, tp_prices,entry_price,atr_values):
    close_price = c.close[c.i, c.col]
    # print("INDEX", c.i)
    # print("COL", c.col)
    # print("high", high[c.i])
    # print('open', open_[c.i])
    # print("close", close_price)
    # print('low', low[c.i])
    # print('position size :', c.position_now)
    # print('cash:', c.cash_now)
    # print('entries:', entries[c.i, c.col])
    # print('sl_prices:', sl_prices[c.i])
    # print('tp_prices:', tp_prices[c.i])
    # print('entry_price:', entry_price[c.i])
    # print()

# if in position 
    if (c.position_now > 0):
        # Check if SL is hit
        # if c.close[c.i-1, c.col] <= sl_prices[c.i]:
        if open_[c.i] <= sl_prices[c.i]:
        # if low[c.i] <= sl_prices[c.i]:
            value = vbt.portfolio.nb.order_nb(
                size=-np.inf,  # Close position
                price=open_[c.i],  # Current closing price
                size_type=SizeType.Amount,
                direction=Direction.LongOnly,
                fees=0.001,
                slippage=0.002)
            # print('sl hit at index',c.i)
            # print('sl order', value)

            return value

        # sl update for long position


        if (entries[c.i-1, c.col] == 5):  
            highest_low = np.max(low[c.i-7:c.i])
            update2 = highest_low - atr_values[c.i] * 0.2
            if update2 > sl_prices[c.i]:
                # print('sl update before',c.i,sl_prices[c.i])
                sl_prices[:]= update2

        #losen sl for less valatility 
        if (entries[c.i-1, c.col] == 10): 
            highest_low = np.max(low[c.i-7:c.i])
            update = highest_low - atr_values[c.i]
            if update > sl_prices[c.i]:
                # print('sl update before',c.i,sl_prices[c.i])
                sl_prices[:]= update
  


    # if not in position search for position to enter
    elif (c.position_now == 0) & (c.i != 0):
        if entries[c.i-1, c.col] == 10:
            sl_prices[:] = np.nan
            entry_price[:] = open_[c.i]
            order = vbt.portfolio.nb.order_nb(
                size=1,  # Adjusted order size
                price=open_[c.i],  # Current closing price
                size_type=SizeType.Percent,  # Specify size type
                direction=Direction.LongOnly,  # Long-only trading
                fees=0.001,  # No fees
                slippage=0.002,  # No slippage
                allow_partial=False,  # Do not allow partial fills
                raise_reject=True  # Raise an error if the order is rejected
            )

            sl_prices[:] = low[c.i] - atr_values[c.i] 
            # print('entry', order)
            return order

    
    return vbt.portfolio.enums.NoOrder


close = df_renamed_close
entries = df_custom_indiator.to_numpy()

open_ = df['open'].to_numpy().flatten()
high = df['high'].to_numpy().flatten()
low = df['low'].to_numpy().flatten()

atr_values = talib.ATR(df['high'], df['low'], df['close'], timeperiod=7).to_numpy().flatten()

# Create an array to store SL prices
sl_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
tp_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
entry_price = np.full(close.shape[0], np.nan)  # Use a 1D array

# Create portfolio
pf = vbt.Portfolio.from_order_func(
    close,           # Price DataFrame
    order_func_nb,
    high,
    low,
    open_,
    entries,    # Order function
    sl_prices,
    tp_prices,
    entry_price,
    atr_values,  # Pass the SL prices array
    init_cash=500  # Initial cash balance
)

# Display some portfolio performance metrics
print("Total Return:", pf.total_return())
print("\nOrder Records:")

pf.orders.records_readable




Total Return: momentum_atr_length  momentum_ema_length  momentum_vola_multiplier  momentum_vol_window
4                    100                  1.3                       4                      0.326415
                                                                    5                      0.178144
                                                                    6                      0.145988
                                                                    7                      0.417273
                                          1.4                       4                      0.294689
                                                                                             ...   
9                    240                  1.7                       7                      0.738451
                                          1.8                       4                      0.171220
                                                                    5                      0.94708

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,"(4, 100, 1.3, 4)",2021-04-11 00:00:00+00:00,0.008340,59889.7404,0.499500,Buy
1,1,"(4, 100, 1.3, 4)",2021-04-18 00:00:00+00:00,0.008340,59875.3094,0.499380,Sell
2,2,"(4, 100, 1.3, 4)",2021-04-28 00:00:00+00:00,0.009041,55124.2284,0.498382,Buy
3,3,"(4, 100, 1.3, 4)",2021-05-05 00:00:00+00:00,0.009041,53089.9074,0.479990,Sell
4,4,"(4, 100, 1.3, 4)",2021-05-07 00:00:00+00:00,0.008477,56510.4954,0.479031,Buy
...,...,...,...,...,...,...,...
128983,128983,"(9, 240, 1.8, 7)",2024-12-13 00:00:00+00:00,0.009187,100202.9058,0.920568,Buy
128984,128984,"(9, 240, 1.8, 7)",2024-12-19 00:00:00+00:00,0.009187,100002.4942,0.918727,Sell
128985,128985,"(9, 240, 1.8, 7)",2024-12-27 00:00:00+00:00,0.009553,95977.4718,0.916891,Buy
128986,128986,"(9, 240, 1.8, 7)",2024-12-30 00:00:00+00:00,0.009553,93552.3204,0.893724,Sell


In [176]:
total_return = pf.total_return()
max_dd = pf.max_drawdown()
return_and_maxdd = pd.concat([total_return, max_dd], axis=1)
return_and_maxdd.sort_values(by='total_return', ascending=False).head(50)



total_return  \
momentum_atr_length momentum_ema_length momentum_vola_multiplier momentum_vol_window                 
7                   140                 1.6                      7                        1.955725   
                    160                 1.6                      7                        1.954450   
6                   140                 1.6                      7                        1.883033   
                    160                 1.6                      7                        1.881790   
5                   140                 1.7                      7                        1.872155   
                    160                 1.7                      7                        1.870916   
9                   140                 1.6                      7                        1.862184   
                    160                 1.6                      7                        1.860949   
6                   140                 1.7                      7                        1.808397   
                    160                 1.7                      7                        1.807186   
8                   140                 1.6                      7                        1.766769   
                    160                 1.6                      7                        1.765575   
9                   140                 1.7                      7                        1.712865   
                    160                 1.7                      7                        1.711695   
8                   140                 1.7                      7                        1.709811   
7                   140                 1.7                      7                        1.709811   
                    160                 1.7                      7                        1.708642   
8                   160                 1.7                      7                        1.708642   
7                   140                 1.6                      6                        1.696839   
                    160                 1.6                      6                        1.695676   
                    180                 1.6                      7                        1.649869   
5                   140                 1.6                      7                        1.636837   
                    160                 1.6                      7                        1.635699   
8                   140                 1.6                      6                        1.616209   
9                   140                 1.6                      6                        1.616209   
8                   160                 1.6                      6                        1.615080   
9                   160                 1.6                      6                        1.615080   
                    140                 1.7                      6                        1.613165   
6                   140                 1.6                      6                        1.612429   
9                   160                 1.7                      6                        1.612037   
6                   160                 1.6                      6                        1.611302   
9                   160                 1.6                      5                        1.606998   
7                   140                 1.8                      6                        1.599233   
                    160                 1.8                      6                        1.598111   
                    200                 1.6                      7                        1.585821   
6                   180                 1.6                      7                        1.584700   
9                   140                 1.8                      6                        1.583154   
                    160                 1.8                      6                        1.582040   
5     

In [177]:
# Access the setup for a specific name, e.g., '(4, 100, 1.0, 4)'
# 8	160	1.8	7	
setup_name = (8, 160, 1.8, 7)
specific_pf = pf[setup_name]
specific_pf.plot().show()



In [178]:
specific_pf2 = pf.select_one(setup_name)
orders = specific_pf2.orders.records_readable
orders

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,96799,"(8, 160, 1.8, 7)",2021-07-31 00:00:00+00:00,0.011812,42287.3058,0.499500,Buy
1,96800,"(8, 160, 1.8, 7)",2021-08-04 00:00:00+00:00,0.011812,38125.1968,0.450337,Sell
2,96801,"(8, 160, 1.8, 7)",2021-08-07 00:00:00+00:00,0.010470,42925.1790,0.449438,Buy
3,96802,"(8, 160, 1.8, 7)",2021-08-31 00:00:00+00:00,0.010470,46887.0380,0.490919,Sell
4,96803,"(8, 160, 1.8, 7)",2021-09-01 00:00:00+00:00,0.010382,47189.6910,0.489938,Buy
...,...,...,...,...,...,...,...
96,96895,"(8, 160, 1.8, 7)",2024-12-12 00:00:00+00:00,0.012662,101319.1338,1.282876,Buy
97,96896,"(8, 160, 1.8, 7)",2024-12-19 00:00:00+00:00,0.012662,100002.4942,1.266205,Sell
98,96897,"(8, 160, 1.8, 7)",2024-12-27 00:00:00+00:00,0.013166,95977.4718,1.263675,Buy
99,96898,"(8, 160, 1.8, 7)",2024-12-30 00:00:00+00:00,0.013166,93552.3204,1.231745,Sell


In [179]:
orders_time_filered = orders[orders['Timestamp'] > '2023-01-01 00:00:00']
orders_time_filered

,Order Id,Column,Timestamp,Size,Price,Fees,Side
20,96819,"(8, 160, 1.8, 7)",2023-01-14 00:00:00+00:00,0.024947,19964.7498,0.498061,Buy
21,96820,"(8, 160, 1.8, 7)",2023-02-07 00:00:00+00:00,0.024947,22714.4800,0.566659,Sell
22,96821,"(8, 160, 1.8, 7)",2023-02-16 00:00:00+00:00,0.023200,24376.2552,0.565527,Buy
23,96822,"(8, 160, 1.8, 7)",2023-02-25 00:00:00+00:00,0.023200,23140.7258,0.536863,Sell
24,96823,"(8, 160, 1.8, 7)",2023-03-02 00:00:00+00:00,0.022629,23677.1598,0.535790,Buy
...,...,...,...,...,...,...,...
96,96895,"(8, 160, 1.8, 7)",2024-12-12 00:00:00+00:00,0.012662,101319.1338,1.282876,Buy
97,96896,"(8, 160, 1.8, 7)",2024-12-19 00:00:00+00:00,0.012662,100002.4942,1.266205,Sell
98,96897,"(8, 160, 1.8, 7)",2024-12-27 00:00:00+00:00,0.013166,95977.4718,1.263675,Buy
99,96898,"(8, 160, 1.8, 7)",2024-12-30 00:00:00+00:00,0.013166,93552.3204,1.231745,Sell
